## <center> **Livrable n°1 : Classification binaire** </center>

‎ 

Réalisé par le **groupe n°2** :
- BERTHO Lucien
- BOSACKI Paul
- GAURE Warren
- GRENOUILLET Théo
- VALLEMONT Hugo


‎

---


### **Sommaire**

1. [Mise en contexte](#contexte)
2. [Objectif du livrable](#objectif)
3. [Importation des bibliothèques](#import)
4. [Chargement et préparation des données](#load)
5. [Exploration et visualisation des données](#exploration)
6. [Configuration de l'environnement](#configuration)
7. [Choix de l'architecture](#architecture)
8. [Réalisation du modèle](#modele)
9.  [Entraînement et évaluation du modèle](#train)
10. [Amélioration du modèle](#amelioration)
11. [Modèle final](#final)
12. [Conclusion](#conclusion)

‎ 

---

### 1. <a id='contexte'>Mise en contexte</a>

L’entreprise TouNum est spécialisée dans la numérisation de documents, qu’il s’agisse de textes ou d’images. Ses services sont particulièrement sollicités par des entreprises cherchant à transformer leur base documentaire papier en fichiers numériques exploitables. Aujourd’hui, TouNum souhaite aller plus loin en enrichissant son offre avec des outils basés sur le Machine Learning.

En effet, certains clients disposent d’un volume considérable de documents à numériser et expriment un besoin croissant pour des solutions de catégorisation automatique. Une telle innovation leur permettrait d’optimiser le traitement et l’exploitation de leurs données numérisées. Toutefois, TouNum ne dispose pas en interne des compétences nécessaires pour concevoir et mettre en place ces technologies.

C’est dans ce cadre que notre équipe de spécialistes en Data Science du CESI est sollicitée. Notre mission consiste à développer une première solution intégrant du captioning automatique : un système capable d’analyser des photographies et de générer une légende descriptive de manière autonome.

Heureusement, TouNum possède déjà plusieurs milliers d’images annotées, ce qui constituera une ressource précieuse pour entraîner les modèles de Machine Learning à partir d’un apprentissage supervisé.

---

### 2. <a id='objectif'>Objectif du livrable</a>

TouNum souhaite automatiser la sélection des photos destinées à l'annotation. Ce livrable propose une méthode de classification basée sur les réseaux de neurones pour filtrer les images qui ne sont pas des photos. La solution reposera sur l'architecture de réseau retenue en fonction des résultats obtenus.

---

### 3. <a id='import'>Importation des bibliothèques</a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import collections
import os

from concurrent.futures import ThreadPoolExecutor, as_completed
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

---

### 4. <a id='load'>Préparation et chargement des données</a>

Une fois les bibliothèques importées, nous pouvons commencer à préparer les données et les charger pour qu'elles puissent être utilisées dans notre pipeline.

#### 4.1. 

Tout d'abord, nous devons nous assurer que les images reçues ne sont pas corrompues ou mal formatées.

In [ ]:
dataset_directory = "dataset_livrable_1/"

In [ ]:
def is_valid_image(path):
    try:
        img_raw = tf.io.read_file(path)
        _ = tf.image.decode_image(img_raw, channels=3)
        return (path, True)
    except Exception:
        return (path, False)

def clean_corrupted_images(directory, extensions=("jpg", "jpeg", "png"), max_workers=8):
    image_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(extensions):
                image_paths.append(os.path.join(root, file))

    print(f"Scan de {len(image_paths)} images dans {directory}...")

    corrupted_count = 0
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(is_valid_image, path) for path in image_paths]
        for future in as_completed(futures):
            path, is_valid = future.result()
            if not is_valid:
                try:
                    os.remove(path)
                    corrupted_count += 1
                except Exception as e:
                    print(f"Erreur suppression {path} : {e}")

    print(f"Terminé : {corrupted_count} image(s) corrompue(s) supprimée(s).")
    

clean_corrupted_images(dataset_directory)

Les images doivent être séparées en deux ensembles : un pour l'entraînement du modèle, l'autre pour son évaluation.

In [ ]:
image_h = 180
image_w = 180
batch_s = 32

train_set = keras.utils.image_dataset_from_directory(
    dataset_directory,
    label_mode = "int",
    batch_size = batch_s,
    image_size = (image_h, image_w),
    seed = 42,
    validation_split = 0.2,
    subset = "training"
)

test_set = keras.utils.image_dataset_from_directory(
    dataset_directory,
    label_mode = "int",
    batch_size = batch_s,
    image_size = (image_h, image_w),
    seed = 42,
    validation_split = 0.2,
    subset = "validation",
)

La sortie indique la présence de **41399** fichiers au total, dont **33120** appartenant au jeu d'entraînement et **8279** au jeu de test.

Une fois les données réparties dans les deux ensembles, nous pouvons afficher le nom des labels.

In [ ]:
class_names = train_set.class_names
print(f"Classes détectées : {class_names}")

Après vérification des classes, nous pouvons vérifier la répartition des données entre elles.

In [ ]:
def print_class_distribution(dataset):
    label_counts = collections.Counter(label.numpy() for _, label in dataset.unbatch())

    classes = {
        0: "peintures",
        1: "photos",
        2: "schémas",
        3: "croquis",
        4: "textes scannés"
    }

    total = sum(label_counts.values())
    print(f"Nombre total d'images : {total}")

    for label, count in label_counts.items():
        class_name = classes.get(label, f"Classe inconnue ({label})")
        print(f"Nombre de {class_name} : {count} ({count / total * 100:.2f}% du total)")

In [ ]:
print("------------------------- TRAIN SET -------------------------")
print_class_distribution(train_set)
print("------------------------- TEST SET --------------------------")
print_class_distribution(test_set)

---

### 5. <a id='exploration'>Exploration et visualisation des données</a>

Une fois les données chargées et les labels modifiés, nous pouvons examiner plus en détail les images.

Nous affichons quelques images pour regarder plus en détail ce à quoi nous avons affaire.

In [ ]:
plt.figure(figsize = (8, 8))
for images, labels in train_set.take(10):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i].numpy()])
        plt.axis("off")

Enfin, nous affichons la taille des données, information pouvant être utile pour gérer les performances de notre modèle.

In [ ]:
images, labels = next(iter(train_set.take(1)))
print(f"Tensor des images : {images.shape}")
print(f"Tensor des labels : {labels.shape}")

---

### 6. <a id='configuration'>Configuration de l'environnement</a>

Pour optimiser les performances des calculs, nous allons configurer les données à l’aide de deux fonctions : `Dataset.cache` et `Dataset.prefetch`.  
- [`Dataset.cache`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#cache) stocke les données en mémoire pour éviter les accès répétés au disque.  
- [`Dataset.prefetch`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#prefetch) permet de traiter un élément en arrière-plan pendant l'entraînement ou l'évaluation.  

En combinant ces techniques, nous réduirons significativement le temps de traitement et la charge computationnelle.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_set = train_set.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
test_set = test_set.cache().prefetch(buffer_size = AUTOTUNE)

---

### 7. <a id='architecture'>Choix de l'architecture</a>

Nous avons choisi une architecture basée sur les **réseaux de neurones convolutifs (CNN)**, particulièrement adaptés à la classification binaire d’images. Contrairement aux réseaux classiques (MLP), qui traitent chaque pixel indépendamment, les CNN exploitent les relations entre pixels pour mieux détecter les motifs et les formes. Grâce à leurs **filtres partagés**, ils réduisent considérablement le nombre de paramètres, rendant l’entraînement plus rapide et limitant le surapprentissage. Ces propriétés en font une solution puissante et efficace pour notre problème.

---

### 8. <a id='modele'>Réalisation du modèle</a>

Maintenant que le choix de l'architecture est fait, nous pouvons commencer à créer le modèle que nous allons utiliser pour classifier les images envoyées par l'entreprise.

Notre modèle sera structuré autour des blocs suivants :  
- Une **couche de rescaling** pour normaliser les valeurs des composantes RGB des pixels dans l'intervalle `[0;1]`.  
- Une **première convolution** avec 16 filtres de taille 3x3 (`Conv2D`), suivie d'un **max pooling** pour réduire la dimension spatiale.  
- Une **seconde convolution** utilisant 32 filtres de taille 3x3.  
- Une **troisième convolution** avec 64 filtres de taille 3x3.  
- Une **transformation en vecteur** via une opération d'aplatissement (`Flatten`).  
- Une **couche dense** de 128 unités pour capturer les caractéristiques extraites.  
- Enfin, une **sortie entièrement connectée** avec 1 unité, correspondant à la classe cible.  

In [ ]:
num_classes = len(class_names)

model = Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(16, (3, 3), padding = 'same', activation = 'relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu'),
    layers.Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    layers.Flatten(),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(num_classes, activation = 'softmax')
])

model.compile(
    optimizer = 'adam',
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ['accuracy']
)

L'optimiseur [`Adam`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) est choisi pour sa capacité d'adaptation et sa rapidité de convergence, combinant les avantages de [`Momentum`](https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/MomentumOptimizer) et [`RMSprop`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/RMSprop).

La fonction de perte [`SparseCategoricalCrossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy) est utilisée car elle est plus efficace en mémoire et bien adaptée à la classification multi-classes.

Le code suivant va permettre d'avoir un résumé du modèle tel qu'il est à ce stade.

In [ ]:
model.summary()

---

### 9. <a id='train'>Entraînement et évaluation du modèle</a>

Avec le modèle créé, nous pouvons désormais procéder à son entraînement et son évaluation avec les ensembles de données à notre disposition.

In [ ]:
epochs = 10
epochs_range = range(epochs)

history = model.fit(
    train_set,
    validation_data = test_set,
    epochs = epochs
)

accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

loss = history.history['loss']
validation_loss = history.history['val_loss']

plt.figure(figsize = (16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label = 'Training Accuracy')
plt.plot(epochs_range, validation_accuracy, label = 'Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, validation_loss, label = 'Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

<b style="color:yellow;">TODO</b>

Analyser les résultats.

In [ ]:
y_true = []
X_test = []

for images, labels in test_set:
    y_true.append(labels)
    X_test.append(images)

X_test = np.concatenate(X_test)
y_true = np.concatenate(y_true)

In [ ]:
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis = 1)

In [ ]:
cm = confusion_matrix(y_true, y_pred)
display = ConfusionMatrixDisplay(cm, display_labels = class_names)
display.plot(cmap = plt.cm.Blues)
plt.title("Matrice de confusion")
plt.xticks(rotation = 45)   # Incliner les étiquettes horizontalement
plt.show()

---

### 10. <a id='amelioration'>Amélioration du modèle</a>

Afin de palier au surapprentissage observé et d’améliorer la généralisation du modèle, plusieurs techniques de régularisation ont été retenues :

- [**Data Augmentation**](#augmentation): Cette technique consister à générer artificiellement de nouvelles images en appliquant des transformations aléatoires aux données existantes. Elle permet d'améliorer la généralisation du modèle en le rendant plus robuste aux variations comme l’orientation, la luminosité ou le zoom.

- [**Dropout**](#dropout) : Cette méthode consiste à désactiver aléatoirement un certain pourcentage de neurones à chaque itération lors de l'entraînement. Cela empêche le modèle de devenir trop dépendant de certaines connexions et encourage l'apprentissage de représentations plus robustes. Une valeur typique se situe entre 0.2 et 0.5 selon la complexité du réseau.

- [**Early-Stopping**](#early-stopping) : Cette technique permet d'arrêter automatiquement l'entraînement lorsque la performance sur l’ensemble de validation commence à se dégrader. Elle évite d’entraîner le modèle trop longtemps, ce qui pourrait mener à un surajustement aux données d’entraînement. Un paramètre clé est la `patience`, qui définit le nombre d’époques d'attente avant d'interrompre l'entraînement si aucune amélioration n'est observée.

- [**Régularisation L1 (Lasso)**](#l1) : L1 applique une pénalité sur la somme des valeurs absolues des poids du modèle. Cela pousse certains poids à devenir exactement nuls, ce qui permet une sparsité dans les connexions et peut aider à réduire l'impact des variables les moins importantes. Elle est particulièrement utile pour éliminer les caractéristiques inutiles dans des modèles fortement paramétrés.

- [**Régularisation L2 (Ridge)**](#l2) : L2 impose une pénalité sur la somme des carrés des poids, ce qui réduit la magnitude des paramètres sans les annuler complètement. Cela permet de contrôler des valeurs extrêmes qui pourraient rendre le modèle trop sensible aux fluctuations des données d’entraînement. Cette méthode est couramment utilisée dans les réseaux convolutifs, où il est essentiel d'éviter des poids trop dominants.

En testant et, potentiellement, combinant ces différentes approches, nous parviendrons à obtenir un modèle plus stable, robuste, et capable de mieux généraliser sur des données non vues.

──────────────────────────────────────────────────

#### 10.1. <a id='augmentation'>Data Augmentation</a>

Pour cette approche, nous allons appliquer des transformations aux données d'entraînement, comme un retournement aléatoire, une rotation de 10% et d’un zoom vertical de 10%. Les données de test restent inchangées pour permettre au modèle de pouvoir faire des prédictions sur un cas réel.

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip(input_shape = (image_h, image_w, 3), mode = 'horizontal_and_vertical'),
    layers.RandomRotation(factor = 0.1, fill_mode = 'nearest'),
    layers.RandomZoom(height_factor = 0.1, fill_mode = 'nearest'),
])

train_set_augmented = train_set.map(lambda x, y: (data_augmentation(x, training = True), y))

In [ ]:
model_with_augmentation = Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(16, (3, 3), padding = 'same', activation = 'relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu'),
    layers.Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    layers.Flatten(),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(num_classes, activation = 'softmax')
])

model_with_augmentation.compile(
    optimizer = 'adam',
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ['accuracy']
)

model_with_augmentation.summary()

Nous pouvons procéder à l'entraînement de cette version du modèle.

In [ ]:
epochs = 10
epochs_range = range(epochs)

history = model_with_augmentation.fit(
    train_set_augmented,
    validation_data = test_set,
    epochs = epochs
)

accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

loss = history.history['loss']
validation_loss = history.history['val_loss']

plt.figure(figsize = (16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label = 'Training Accuracy')
plt.plot(epochs_range, validation_accuracy, label = 'Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, validation_loss, label = 'Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

&nbsp;

**Notes lors des tests**
- Méthode efficace pour gérer le surapprentissage
- Les courbes d'accuracy augmentent ensemble de manière cohérente
- Pas de divergence apparente
- Très bonne capacité de généralisation vu que la courbe de Validation Accuracy est souvent au-dessus de celle de la Training Accuracy

In [ ]:
y_pred_proba = model_with_augmentation.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis = 1)

In [ ]:
cm = confusion_matrix(y_true, y_pred)
display = ConfusionMatrixDisplay(cm, display_labels = class_names)
display.plot(cmap = plt.cm.Blues)
plt.title("Matrice de confusion")
plt.xticks(rotation = 45)
plt.show()

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

──────────────────────────────────────────────────

#### 10.2. <a id='dropout'>Dropout</a>

Pour cette approche, nous ajoutons un Dropout de 25% après le MaxPooling et un Dropout de 50% après la couche Dense, en s'inspirant de l’approche de [Keras pour le dataset MNIST](https://github.com/keras-team/keras/blob/keras-2/examples/mnist_cnn.py). Le premier limite la dépendance aux caractéristiques extraites, tandis que le second réduit le surajustement dans la partie dense.

In [ ]:
model_with_dropout = Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(16, (3, 3), padding = 'same', activation = 'relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu'),
    layers.Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation = 'softmax')
])

model_with_dropout.compile(
    optimizer = 'adam',
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ['accuracy']
)

model_with_dropout.summary()

Nous pouvons procéder à l'entraînement de cette version du modèle.

In [ ]:
epochs = 10
epochs_range = range(epochs)

history = model_with_dropout.fit(
    train_set,
    validation_data = test_set,
    epochs = epochs
)

accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

loss = history.history['loss']
validation_loss = history.history['val_loss']

plt.figure(figsize = (16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label = 'Training Accuracy')
plt.plot(epochs_range, validation_accuracy, label = 'Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, validation_loss, label = 'Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

&nbsp;

**Notes lors des tests**
- Apprend trop vite (Traning Accuracy > 95%) mais ne généralise pas mieux pour autant (Validation Loss en hausse)
- La Validation Accuracy se stabilise au bout de la 6ème époque
- Écart significatif entre la Traning Accuracy et la Validation Accuracy => surapprentissage
- Le Dropout seul semble ne pas suffire

In [ ]:
y_pred_proba = model_with_dropout.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis = 1)

In [ ]:
cm = confusion_matrix(y_true, y_pred)
display = ConfusionMatrixDisplay(cm, display_labels = class_names)
display.plot(cmap = plt.cm.Blues)
plt.title("Matrice de confusion")
plt.xticks(rotation = 45)
plt.show()

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

──────────────────────────────────────────────────

#### 10.3. <a id='early-stopping'>Early Stopping</a>

Pour cette approche, nous ajoutons un Early Stopping basé sur la perte de validation, avec une patience de 5 époques, afin d’interrompre l’entraînement dès que le modèle cesse de s’améliorer et de conserver les meilleurs poids.

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor = 'val_accuracy',
    patience = 2,
    mode = 'max',
    restore_best_weights = True
)

In [ ]:
model_with_early_stopping = Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(16, (3, 3), padding = 'same', activation = 'relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu'),
    layers.Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    layers.Flatten(),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(num_classes, activation = 'softmax')
])

model_with_early_stopping.compile(
    optimizer = 'adam',
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ['accuracy']
)

model_with_early_stopping.summary()

Nous pouvons procéder à l'entraînement de cette version du modèle.

In [ ]:
epochs = 20

history = model_with_early_stopping.fit(
    train_set,
    validation_data = test_set,
    epochs = epochs,
    callbacks = [early_stopping]
)

accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

epochs_range = range(len(accuracy))

loss = history.history['loss']
validation_loss = history.history['val_loss']

plt.figure(figsize = (16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label = 'Training Accuracy')
plt.plot(epochs_range, validation_accuracy, label = 'Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, validation_loss, label = 'Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

&nbsp;

**Notes lors des tests**
- Apprend trop vite (Traning Accuracy > 95%) mais ne généralise pas mieux pour autant
- Traning Accuracy proche de 100% à la fin + écart avec Validation Accuracy => surapprentissage
- La Validation Loss augmente, ce qui est un autre signe du surapprentissage
- Méthode insuffisante à elle seule pour prévenir l'overfitting

In [ ]:
y_pred_proba = model_with_early_stopping.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis = 1)

In [ ]:
cm = confusion_matrix(y_true, y_pred)
display = ConfusionMatrixDisplay(cm, display_labels = class_names)
display.plot(cmap = plt.cm.Blues)
plt.title("Matrice de confusion")
plt.xticks(rotation = 45)
plt.show()

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

──────────────────────────────────────────────────

<b style="color:yellow;">TODO</b>

Écrire un texte permettant de faire le point sur les tests individuels des méthodes de régularisation et établir une liste de combinaisons en se basant sur les résultats observés

──────────────────────────────────────────────────

#### 10.6. Combinaison #1 : Data Augmentation & Dropout

Nous combinons ici Data Augmentation et Dropout afin de tester leur impact conjoint sur la régularisation du modèle.

In [ ]:
model_with_augmentation_dropout = Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(16, (3, 3), padding = 'same', activation = 'relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu'),
    layers.Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation = 'softmax')
])

model_with_augmentation_dropout.compile(
    optimizer = 'adam',
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ['accuracy']
)

model_with_augmentation_dropout.summary()

Nous pouvons procéder à l'entraînement de cette version du modèle.

In [ ]:
epochs = 10
epochs_range = range(epochs)

history = model_with_augmentation_dropout.fit(
    train_set_augmented,
    validation_data = test_set,
    epochs = epochs
)

accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

loss = history.history['loss']
validation_loss = history.history['val_loss']

plt.figure(figsize = (16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label = 'Training Accuracy')
plt.plot(epochs_range, validation_accuracy, label = 'Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, validation_loss, label = 'Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

&nbsp;

**Notes lors des tests**
- Très très bons résultats
- Apprentissage stable et aucune trace d'overfitting
- Absence de "bruits" (pics dans les graphiques) donc régularisation efficace
- **À garder !**

In [ ]:
y_pred_proba = model_with_augmentation_dropout.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis = 1)

In [ ]:
cm = confusion_matrix(y_true, y_pred)
display = ConfusionMatrixDisplay(cm, display_labels = class_names)
display.plot(cmap = plt.cm.Blues)
plt.title("Matrice de confusion")
plt.xticks(rotation = 45)
plt.show()

<b style="color:yellow;">TODO</b>

Analyser les résultats obtenus.

---

### 11. <a id='final'>Modèle final</a>

<b style="color:yellow;">TODO</b>

Indiquer et justifier le choix du modèle final à l'aide des résultats observés. Faire la passe sur ses caractéristiques, à savoir :
- Paramètres
- Fonction de perte
- Algorithme d'optimisation utilisé pour l'entraînement

Inclure un schéma du modèle réalisé grâce à cet [outil](https://alexlenail.me/NN-SVG/LeNet.html).

---

### 12. <a id='conclusion'>Conclusion</a>

<b style="color:yellow;">TODO</b>

Écrire la conclusion.

---